In [36]:
import torch
import torch.nn as nn
# from transformers.modeling_bert import BertPreTrainedModel, BertModel, BertConfig
from transformers.models.bert.modeling_bert import BertPreTrainedModel
from transformers import AutoModel, AutoTokenizer, BertTokenizer


# from torchcrf import CRF
from TorchCRF import CRF
# from .module import IntentClassifier, SlotClassifier

In [9]:
class IntentClassifier(nn.Module):
    def __init__(self, input_dim, num_intent_labels, dropout_rate=0.):
        super(IntentClassifier, self).__init__()
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(input_dim, num_intent_labels)

    def forward(self, x):
        x = self.dropout(x)
        return self.linear(x)


class SlotClassifier(nn.Module):
    def __init__(self, input_dim, num_slot_labels, dropout_rate=0.):
        super(SlotClassifier, self).__init__()
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(input_dim, num_slot_labels)

    def forward(self, x):
        x = self.dropout(x)
        return self.linear(x)

In [73]:
class JointBERT(BertPreTrainedModel):
    def __init__(self, config, args, intent_label_lst, slot_label_lst):
        super(JointBERT, self).__init__(config)
        self.args = args
        self.config = config
        self.num_intent_labels = len(intent_label_lst)
        self.num_slot_labels = len(slot_label_lst)
            
        self.bert = AutoModel.from_pretrained('JointBERT-snips',config=self.config,local_files_only=True)
#         self.bert = BertModel(config=config)  # Load pretrained bert

        self.intent_classifier = IntentClassifier(config.hidden_size, self.num_intent_labels, args.dropout_rate)
        self.slot_classifier = SlotClassifier(config.hidden_size, self.num_slot_labels, args.dropout_rate)

        if args.use_crf:
            self.crf = CRF(num_tags=self.num_slot_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, token_type_ids, intent_label_ids, slot_labels_ids):
        outputs = self.bert(input_ids, attention_mask=attention_mask,
                            token_type_ids=token_type_ids)  # sequence_output, pooled_output, (hidden_states), (attentions)
        sequence_output = outputs[0]
        pooled_output = outputs[1]  # [CLS]

        intent_logits = self.intent_classifier(pooled_output)
        slot_logits = self.slot_classifier(sequence_output)

        total_loss = 0
        # 1. Intent Softmax
        if intent_label_ids is not None:
            if self.num_intent_labels == 1:
                intent_loss_fct = nn.MSELoss()
                intent_loss = intent_loss_fct(intent_logits.view(-1), intent_label_ids.view(-1))
            else:
                intent_loss_fct = nn.CrossEntropyLoss()
                intent_loss = intent_loss_fct(intent_logits.view(-1, self.num_intent_labels), intent_label_ids.view(-1))
            total_loss += intent_loss

        # 2. Slot Softmax
        if slot_labels_ids is not None:
            if self.args.use_crf:
                slot_loss = self.crf(slot_logits, slot_labels_ids, mask=attention_mask.byte(), reduction='mean')
                slot_loss = -1 * slot_loss  # negative log-likelihood
            else:
                slot_loss_fct = nn.CrossEntropyLoss(ignore_index=self.args.ignore_index)
                # Only keep active parts of the loss
                if attention_mask is not None:
                    active_loss = attention_mask.view(-1) == 1
                    active_logits = slot_logits.view(-1, self.num_slot_labels)[active_loss]
                    active_labels = slot_labels_ids.view(-1)[active_loss]
                    slot_loss = slot_loss_fct(active_logits, active_labels)
                else:
                    slot_loss = slot_loss_fct(slot_logits.view(-1, self.num_slot_labels), slot_labels_ids.view(-1))
            total_loss += self.args.slot_loss_coef * slot_loss

        outputs = ((intent_logits, slot_logits),) + outputs[2:]  # add hidden states and attention if they are here

        outputs = (total_loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions) # Logits is a tuple of intent and slot logits

In [ ]:
## Debug

In [76]:
# import argparse
# from transformers import AutoModel, AutoTokenizer, BertTokenizer
# from ipynb.fs.defs.data_loader import load_and_cache_examples,convert_examples_to_features,JointProcessor,InputFeatures,InputExample


# import logging


# def init_logger():
#     logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
#                         datefmt='%m/%d/%Y %H:%M:%S',
#                         level=logging.INFO)

# if __name__ == '__main__':
#     init_logger()
#     parser = argparse.ArgumentParser()

#     # parser.add_argument("--task", default=None, required=True, type=str, help="The name of the task to train")
#     # parser.add_argument("--model_dir", default=None, required=True, type=str, help="Path to save, load model")
#     parser.add_argument("--data_dir", default="./Dataset", type=str, help="The input data dir")
#     parser.add_argument("--intent_label_file", default="intent_label.txt", type=str, help="Intent Label file")
#     parser.add_argument("--slot_label_file", default="slot_label.txt", type=str, help="Slot Label file")

#     #parser.add_argument("--model_type", default="bert", type=str, help="Model type selected in the list: " + ", ".join(MODEL_CLASSES.keys()))

#     parser.add_argument('--seed', type=int, default=1234, help="random seed for initialization")
#     parser.add_argument("--train_batch_size", default=32, type=int, help="Batch size for training.")
#     parser.add_argument("--eval_batch_size", default=64, type=int, help="Batch size for evaluation.")
#     parser.add_argument("--max_seq_len", default=50, type=int, help="The maximum total input sequence length after tokenization.")
#     parser.add_argument("--learning_rate", default=5e-5, type=float, help="The initial learning rate for Adam.")
#     parser.add_argument("--num_train_epochs", default=10.0, type=float, help="Total number of training epochs to perform.")
#     parser.add_argument("--weight_decay", default=0.0, type=float, help="Weight decay if we apply some.")
#     parser.add_argument('--gradient_accumulation_steps', type=int, default=1,
#                         help="Number of updates steps to accumulate before performing a backward/update pass.")
#     parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
#     parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
#     parser.add_argument("--max_steps", default=-1, type=int, help="If > 0: set total number of training steps to perform. Override num_train_epochs.")
#     parser.add_argument("--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps.")
#     parser.add_argument("--dropout_rate", default=0.1, type=float, help="Dropout for fully-connected layers")

#     parser.add_argument('--logging_steps', type=int, default=200, help="Log every X updates steps.")
#     parser.add_argument('--save_steps', type=int, default=200, help="Save checkpoint every X updates steps.")

#     parser.add_argument("--do_train", action="store_true", help="Whether to run training.")
#     parser.add_argument("--do_eval", action="store_true", help="Whether to run eval on the test set.")
#     parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")

#     parser.add_argument("--ignore_index", default=0, type=int,
#                         help='Specifies a target value that is ignored and does not contribute to the input gradient')
    
#     parser.add_argument("--model_name_or_path", default =  'bert-base-uncased')
#     parser.add_argument("--use_crf", action="store_true", help="Whether to use CRF")
#     parser.add_argument('--slot_loss_coef', type=float, default=1.0, help='Coefficient for the slot loss.')
    
#     parser.add_argument('-f')
#     args = parser.parse_args()
    
#     args.model_name_or_path = 'bert-base-uncased'
    
#     tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#     train_dataset = load_and_cache_examples(args, tokenizer, mode="train")
#     dev_dataset = load_and_cache_examples(args, tokenizer, mode="val")
#     test_dataset = load_and_cache_examples(args, tokenizer, mode="test")
    

In [77]:
# import json
# from transformers import AutoConfig

# def get_slot_labels():
#     path = './Dataset/slot_label.txt'
#     lst = [label.strip() for label in open(path, 'r', encoding='utf-8')]
#     return lst

# def get_intent_labels():
#     path = './Dataset/intent_label.txt'
#     lst = [label.strip() for label in open(path, 'r', encoding='utf-8')]

#     return [lst[0] for i in range(len(lst))]

# # with open('JointBERT-snips/config.json') as config_file:
# #         config = json.load(config_file) 
            
# config = AutoConfig.from_pretrained('JointBERT-snips/config.json')
# JB = JointBERT(config=config,
#                   args=args,
#                   intent_label_lst=get_intent_labels(),
#                   slot_label_lst=get_slot_labels())

In [48]:
# import os
# import logging
# import json
# from tqdm import tqdm, trange

# import numpy as np
# import torch
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# from transformers import BertConfig, AdamW, get_linear_schedule_with_warmup
# from transformers import AutoModel, AutoTokenizer, BertTokenizer

In [78]:
# def get_slot_labels():
#     path = './Dataset/slot_label.txt'
#     lst = [label.strip() for label in open(path, 'r', encoding='utf-8')]
#     return lst

# def get_intent_labels():
#     path = './Dataset/intent_label.txt'
#     lst = [label.strip() for label in open(path, 'r', encoding='utf-8')]

#     return [lst[0] for i in range(len(lst))]

In [75]:
# class Trainer(object):
#     def __init__(self, args, train_dataset=None, val_dataset=None, test_dataset=None):
#         self.args = args
#         self.train_dataset = train_dataset
#         self.val_dataset = val_dataset
#         self.test_dataset = test_dataset

#         self.intent_label_lst = get_intent_labels()#!!!!!!!!!!!!!!!!!1
#         self.slot_label_lst = get_slot_labels()#!!!!!!!!!!!!!!!!!!!!!!!
        
#         # Use cross entropy ignore index as padding label id so that only real label ids contribute to the loss later
#         self.pad_token_label_id = args.ignore_index
        

# #         self.config_class, self.model_class, _ = MODEL_CLASSES[args.model_type]
# #         self.config = self.config_class.from_pretrained(args.model_name_or_path, finetuning_task=args.task)
# #         self.model = self.model_class.from_pretrained(args.model_name_or_path,
# #                                                       config=self.config,
# #                                                       args=args,
# #                                                       intent_label_lst=self.intent_label_lst,
# #                                                       slot_label_lst=self.slot_label_lst)
# #         with open('JointBERT-snips/config.json') as config_file:
# #             self.config = json.load(config_file) 
#         self.config = AutoConfig.from_pretrained('JointBERT-snips/config.json',local_files_only=True)
#         self.model = JointBERT(config=self.config,
#                                   args=self.args,
#                                   intent_label_lst=self.intent_label_lst,
#                                   slot_label_lst=self.slot_label_lst)
#         print(self.model)

            
# #         self.model = AutoModel.from_pretrained('JointBERT-snips',
# #                                                       config=self.config,local_files_only=True)#,


#         # GPU or CPU
#         self.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
#         self.model.to(self.device)

#     def train(self):
#         train_sampler = RandomSampler(self.train_dataset)
#         train_dataloader = DataLoader(self.train_dataset, sampler=train_sampler, batch_size=self.args.train_batch_size)

#         if self.args.max_steps > 0:
#             t_total = self.args.max_steps
#             self.args.num_train_epochs = self.args.max_steps // (len(train_dataloader) // self.args.gradient_accumulation_steps) + 1
#         else:
#             t_total = len(train_dataloader) // self.args.gradient_accumulation_steps * self.args.num_train_epochs

#         # Prepare optimizer and schedule (linear warmup and decay)
#         no_decay = ['bias', 'LayerNorm.weight']
#         optimizer_grouped_parameters = [
#             {'params': [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
#              'weight_decay': self.args.weight_decay},
#             {'params': [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
#         ]
#         optimizer = AdamW(optimizer_grouped_parameters, lr=self.args.learning_rate, eps=self.args.adam_epsilon)
#         scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=self.args.warmup_steps, num_training_steps=t_total)

#         # Train!
# #         init_logger()
# #         logger.info("***** Running training *****")
# #         logger.info("  Num examples = %d", len(self.train_dataset))
# #         logger.info("  Num Epochs = %d", self.args.num_train_epochs)
# #         logger.info("  Total train batch size = %d", self.args.train_batch_size)
# #         logger.info("  Gradient Accumulation steps = %d", self.args.gradient_accumulation_steps)
# #         logger.info("  Total optimization steps = %d", t_total)
# #         logger.info("  Logging steps = %d", self.args.logging_steps)
# #         logger.info("  Save steps = %d", self.args.save_steps)

#         global_step = 0
#         tr_loss = 0.0
#         self.model.zero_grad()

#         train_iterator = trange(int(self.args.num_train_epochs), desc="Epoch")

#         for _ in train_iterator:
#             epoch_iterator = tqdm(train_dataloader, desc="Iteration")
#             for step, batch in enumerate(epoch_iterator):
#                 ####################################################################################################
#                 self.model.train()
#                 batch = tuple(t.to(self.device) for t in batch)  # GPU or CPU

#                 inputs = {'input_ids': batch[0],
#                           'attention_mask': batch[1],
#                           'intent_label_ids': batch[3],
#                           'slot_labels_ids': batch[4]}
#                 # if self.args.model_type != 'distilbert':
#                 inputs['token_type_ids'] = batch[2]
#                 outputs = self.model(**inputs)
#                 loss = outputs[0]

#                 if self.args.gradient_accumulation_steps > 1:
#                     loss = loss / self.args.gradient_accumulation_steps

#                 loss.backward()

#                 tr_loss += loss.item()
#                 if (step + 1) % self.args.gradient_accumulation_steps == 0:
#                     torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.args.max_grad_norm)

#                     optimizer.step()
#                     scheduler.step()  # Update learning rate schedule
#                     self.model.zero_grad()
#                     global_step += 1

#                     if self.args.logging_steps > 0 and global_step % self.args.logging_steps == 0:
#                         self.evaluate("val")

#                     if self.args.save_steps > 0 and global_step % self.args.save_steps == 0:
#                         self.save_model()

#                 if 0 < self.args.max_steps < global_step:
#                     epoch_iterator.close()
#                     break

#             if 0 < self.args.max_steps < global_step:
#                 train_iterator.close()
#                 break

#         return global_step, tr_loss / global_step

#     def evaluate(self, mode):
#         if mode == 'test':
#             dataset = self.test_dataset
#         elif mode == 'val':
#             dataset = self.val_dataset
#         else:
#             raise Exception("Only val and test dataset available")

#         eval_sampler = SequentialSampler(dataset)
#         eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=self.args.eval_batch_size)

#         # Eval!
# #         logger.info("***** Running evaluation on %s dataset *****", mode)
# #         logger.info("  Num examples = %d", len(dataset))
# #         logger.info("  Batch size = %d", self.args.eval_batch_size)
#         eval_loss = 0.0
#         nb_eval_steps = 0
# #         intent_preds = None
#         slot_preds = None
#         out_intent_label_ids = None
#         out_slot_labels_ids = None

#         self.model.eval()

#         for batch in tqdm(eval_dataloader, desc="Evaluating"):
#             batch = tuple(t.to(self.device) for t in batch)
#             with torch.no_grad():
#                 inputs = {'input_ids': batch[0],
#                           'attention_mask': batch[1],
#                           'intent_label_ids': batch[3],
#                           'slot_labels_ids': batch[4]}
#                 #if self.args.model_type != 'distilbert':
#                 inputs['token_type_ids'] = batch[2]
#                 outputs = self.model(**inputs)
#                 tmp_eval_loss, (intent_logits, slot_logits) = outputs[:2]

#                 eval_loss += tmp_eval_loss.mean().item()
#             nb_eval_steps += 1

#             # Intent prediction
#             if intent_preds is None:
#                 intent_preds = intent_logits.detach().cpu().numpy()
#                 out_intent_label_ids = inputs['intent_label_ids'].detach().cpu().numpy()
#             else:
#                 intent_preds = np.append(intent_preds, intent_logits.detach().cpu().numpy(), axis=0)
#                 out_intent_label_ids = np.append(
#                     out_intent_label_ids, inputs['intent_label_ids'].detach().cpu().numpy(), axis=0)

#             # Slot prediction
#             if slot_preds is None:
#                 if self.args.use_crf:
#                     # decode() in `torchcrf` returns list with best index directly
#                     slot_preds = np.array(self.model.crf.decode(slot_logits))
#                 else:
#                     slot_preds = slot_logits.detach().cpu().numpy()

#                 out_slot_labels_ids = inputs["slot_labels_ids"].detach().cpu().numpy()
#             else:
#                 if self.args.use_crf:
#                     slot_preds = np.append(slot_preds, np.array(self.model.crf.decode(slot_logits)), axis=0)
#                 else:
#                     slot_preds = np.append(slot_preds, slot_logits.detach().cpu().numpy(), axis=0)

#                 out_slot_labels_ids = np.append(out_slot_labels_ids, inputs["slot_labels_ids"].detach().cpu().numpy(), axis=0)

#         eval_loss = eval_loss / nb_eval_steps
#         results = {
#             "loss": eval_loss
#         }

#         # Intent result
#         intent_preds = np.argmax(intent_preds, axis=1)

#         # Slot result
#         if not self.args.use_crf:
#             slot_preds = np.argmax(slot_preds, axis=2)
#         slot_label_map = {i: label for i, label in enumerate(self.slot_label_lst)}
#         out_slot_label_list = [[] for _ in range(out_slot_labels_ids.shape[0])]
#         slot_preds_list = [[] for _ in range(out_slot_labels_ids.shape[0])]

#         for i in range(out_slot_labels_ids.shape[0]):
#             for j in range(out_slot_labels_ids.shape[1]):
#                 if out_slot_labels_ids[i, j] != self.pad_token_label_id:
#                     out_slot_label_list[i].append(slot_label_map[out_slot_labels_ids[i][j]])
#                     slot_preds_list[i].append(slot_label_map[slot_preds[i][j]])

#         total_result = compute_metrics(intent_preds, out_intent_label_ids, slot_preds_list, out_slot_label_list)

#         results.update(total_result)

#         logger.info("***** Eval results *****")
#         for key in sorted(results.keys()):
#             logger.info("  %s = %s", key, str(results[key]))

#         return results

#     def save_model(self):
#         # Save model checkpoint (Overwrite)
#         if not os.path.exists(self.args.model_dir):
#             os.makedirs(self.args.model_dir)
#         model_to_save = self.model.module if hasattr(self.model, 'module') else self.model
#         model_to_save.save_pretrained(self.args.model_dir)

#         # Save training arguments together with the trained model
#         torch.save(self.args, os.path.join(self.args.model_dir, 'training_args.bin'))
#         logger.info("Saving model checkpoint to %s", self.args.model_dir)

#     def load_model(self):
#         # Check whether model exists
#         if not os.path.exists(self.args.model_dir):
#             raise Exception("Model doesn't exists! Train first!")

#         try:
#             self.model = JointBERT(config=self.config,
#                                   args=self.args,
#                                   intent_label_lst=self.intent_label_lst,
#                                   slot_label_lst=self.slot_label_lst)
# #                                                           
#             self.model.to(self.device)
#             logger.info("***** Model Loaded *****")
#         except:
#             raise Exception("Some model files might be missing...")

In [79]:
# from transformers import logging
# logging.set_verbosity_error()

# trainer = Trainer(args, train_dataset, dev_dataset, test_dataset)
# trainer.train()